# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{25233834590: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #tensors:1>

In [14]:
x.id_at_location

50844634909

In [15]:
x.id

85673375777

In [16]:
x.owner

<VirtualWorker id:me #tensors:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:85673375777 -> bob:50844634909]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [42]:
bob = sy.VirtualWorker(hook, id="bob")

In [43]:
alice = sy.VirtualWorker(hook, id="alice")

In [44]:
x = th.tensor([1,2,3,4,5])

In [49]:
x_ptr = x.send(bob,alice)

In [46]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [50]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:53351756202 -> bob:67518217082]
	-> [PointerTensor | me:35120615237 -> alice:4581295535]

In [41]:
x.get(sum_results=True)

AttributeError: 'Tensor' object has no attribute 'child'

# Lesson: Introducing Remote Arithmetic

In [51]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [52]:
x

(Wrapper)>[PointerTensor | me:29783770901 -> bob:37649409713]

In [53]:
y

(Wrapper)>[PointerTensor | me:26726799436 -> bob:64917840671]

In [54]:
z = x + y

In [55]:
z

(Wrapper)>[PointerTensor | me:97093566714 -> bob:24592866120]

In [56]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [57]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:11332908298 -> bob:38510455218]

In [58]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [60]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [61]:
z = (x + y).sum()

In [62]:
z.backward()

(Wrapper)>[PointerTensor | me:32100674381 -> bob:32874880811]

In [63]:
x = x.get()

In [64]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [65]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [112]:
input = th.tensor([[1.0,1.0],[0,1.0],[1,0],[0,0]], requires_grad = True).send(bob)

In [113]:
target = th.tensor([[1.0],[0],[1],[0]], requires_grad = True).send(bob)

In [114]:
weights = th.tensor([[0.],[0.]],requires_grad = True).send(bob)

In [115]:
for i in range(20):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad *= 0
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


# Lesson: Garbage Collection and Common Errors


In [116]:
bob = bob.clear_objects()

In [117]:
bob._objects

{}

In [118]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [119]:
bob._objects

{95443330879: tensor([1, 2, 3, 4, 5])}

In [120]:
del x

In [121]:
bob._objects

{}

In [122]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [123]:
bob._objects

{1803400248: tensor([1, 2, 3, 4, 5])}

In [131]:
x = "asdf"

In [132]:
bob._objects

{28959675359: tensor([1, 2, 3, 4, 5])}

In [133]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [134]:
x

(Wrapper)>[PointerTensor | me:99693876309 -> bob:4019539187]

In [135]:
bob._objects

{28959675359: tensor([1, 2, 3, 4, 5]), 4019539187: tensor([1, 2, 3, 4, 5])}

In [136]:
x = "asdf"

In [137]:
bob._objects

{28959675359: tensor([1, 2, 3, 4, 5]), 4019539187: tensor([1, 2, 3, 4, 5])}

In [138]:
del x

In [139]:
bob._objects

{28959675359: tensor([1, 2, 3, 4, 5]), 4019539187: tensor([1, 2, 3, 4, 5])}

In [140]:
bob = bob.clear_objects()
bob._objects

{}

In [141]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [142]:
bob._objects

{30683115053: tensor([1, 2, 3, 4, 5])}

In [143]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [144]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:89350493257 -> bob:26832341266]
Tensor B: tensor([1, 1, 1, 1, 1])

In [145]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [146]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [147]:
from torch import nn, optim

In [148]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [149]:
# A Toy Model
model = nn.Linear(2,1)

In [150]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [85]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [151]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [152]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [153]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [154]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [155]:
train()

tensor(0.9034, requires_grad=True)
tensor(0.1243, requires_grad=True)
tensor(0.0929, requires_grad=True)
tensor(0.0873, requires_grad=True)
tensor(0.0453, requires_grad=True)
tensor(0.0509, requires_grad=True)
tensor(0.0261, requires_grad=True)
tensor(0.0293, requires_grad=True)
tensor(0.0153, requires_grad=True)
tensor(0.0169, requires_grad=True)
tensor(0.0089, requires_grad=True)
tensor(0.0097, requires_grad=True)
tensor(0.0052, requires_grad=True)
tensor(0.0056, requires_grad=True)
tensor(0.0031, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(8.6277e-05, r

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [176]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [177]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [178]:
x = x.send(alice)

In [179]:
bob._objects

{93836708038: tensor([1, 2, 3, 4, 5])}

In [180]:
alice._objects

{14454603410: (Wrapper)>[PointerTensor | alice:14454603410 -> bob:93836708038]}

In [181]:
y = x + x

In [182]:
y

(Wrapper)>[PointerTensor | me:15360440312 -> alice:9759326976]

In [183]:
bob._objects

{93836708038: tensor([1, 2, 3, 4, 5]),
 60783380857: tensor([ 2,  4,  6,  8, 10])}

In [184]:
alice._objects

{14454603410: (Wrapper)>[PointerTensor | alice:14454603410 -> bob:93836708038],
 9759326976: (Wrapper)>[PointerTensor | alice:9759326976 -> bob:60783380857]}

In [185]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [187]:
bob._objects

{98893214848: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{41670494972: (Wrapper)>[PointerTensor | alice:41670494972 -> bob:98893214848]}

In [189]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:41670494972 -> bob:98893214848]

In [190]:
bob._objects

{98893214848: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [194]:
bob._objects

{}

In [195]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [196]:
bob._objects

{11640202963: tensor([1, 2, 3, 4, 5])}

In [197]:
alice._objects

{33395255783: (Wrapper)>[PointerTensor | alice:33395255783 -> bob:11640202963]}

In [198]:
del x

In [199]:
bob._objects

{}

In [200]:
alice._objects

{}

In [201]:
##use this pointer chain to orchestrate the data between workers

# Lesson: Pointer Chain Operations

In [227]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [229]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [230]:
bob._objects

{23684505451: tensor([1, 2, 3, 4, 5])}

In [231]:
alice._objects

{}

In [232]:
x.move(alice)

(Wrapper)>[PointerTensor | me:47457531254 -> alice:64646012673]

In [233]:
bob._objects

{}

In [234]:
alice._objects

{64646012673: tensor([1, 2, 3, 4, 5])}

In [214]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [215]:
bob._objects

{73444622404: tensor([1, 2, 3, 4, 5])}

In [216]:
alice._objects

{37157096612: (Wrapper)>[PointerTensor | alice:37157096612 -> bob:73444622404]}

In [217]:
x.remote_get()

(Wrapper)>[PointerTensor | me:8125438740 -> alice:37157096612]

In [218]:
bob._objects

{}

In [219]:
alice._objects

{37157096612: tensor([1, 2, 3, 4, 5])}

In [220]:
x.move(bob)

(Wrapper)>[PointerTensor | me:77022107055 -> bob:8125438740]

In [221]:
x

(Wrapper)>[PointerTensor | me:77022107055 -> bob:8125438740]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

# Section Project:

For the final project for this section, you're going to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form

In [244]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data/', train=True, download=True, transform=None)

RuntimeError: 

for operator (Tensor 0) -> Tensor:
expected a value of type Tensor for argument '0' but found (Tensor, Tensor, Tensor, Tensor)
    gt_heights = reference_boxes_y2 - reference_boxes_y1
    gt_ctr_x = reference_boxes_x1 + 0.5 * gt_widths
    gt_ctr_y = reference_boxes_y1 + 0.5 * gt_heights

    targets_dx = wx * (gt_ctr_x - ex_ctr_x) / ex_widths
    targets_dy = wy * (gt_ctr_y - ex_ctr_y) / ex_heights
    targets_dw = ww * torch.log(gt_widths / ex_widths)
    targets_dh = wh * torch.log(gt_heights / ex_heights)

    targets = torch.cat((targets_dx, targets_dy, targets_dw, targets_dh), dim=1)
                         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    return targets
:
    gt_heights = reference_boxes_y2 - reference_boxes_y1
    gt_ctr_x = reference_boxes_x1 + 0.5 * gt_widths
    gt_ctr_y = reference_boxes_y1 + 0.5 * gt_heights

    targets_dx = wx * (gt_ctr_x - ex_ctr_x) / ex_widths
    targets_dy = wy * (gt_ctr_y - ex_ctr_y) / ex_heights
    targ

In [239]:
len(mnist_trainset)

NameError: name 'mnist_trainset' is not defined

In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())